# Importing all Modules and importing the data

In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import SVHN
from sklearn.model_selection import train_test_split
import numpy as np

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
num_classes = 10
batch_size = 64
learning_rate = 0.001
num_epochs = 10

# Load SVHN dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = SVHN(root='./data', split='train', transform=transform, download=True)
test_dataset = SVHN(root='./data', split='test', transform=transform, download=True)

Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat


# Splitting the Data

In [2]:
train_dataset, _ = train_test_split(train_dataset, test_size=0.75, random_state=42)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


# Training LeCun Net

In [3]:
models = {
    'LeNet-5': torchvision.models.resnet18(pretrained=True)}

/home/dse313/miniconda3/envs/dse316_assignment/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/dse313/miniconda3/envs/dse316_assignment/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
for model_name, model in models.items():
    model.fc = nn.Linear(512, num_classes)
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    # Train the model
    print(f"Training {model_name}...")
    for epoch in range(num_epochs):
        model.train()
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

Training LeNet-5...


# Testing the LeNet-5 Model

In [5]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy of {model_name}: {100 * correct / total}%")

Accuracy of LeNet-5: 89.98540258143824%


# Training the AlexNet Model

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transforms
transform = transforms.Compose([
    transforms.Resize((227, 227)),  # Resize images to fit AlexNet input size
    transforms.ToTensor(),  # Convert images to tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize images
])

# Load SVHN dataset
train_dataset = datasets.SVHN(root='./data', split='train', download=True, transform=transform)
test_dataset = datasets.SVHN(root='./data', split='test', download=True, transform=transform)

# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Load pre-trained AlexNet
model = torchvision.models.alexnet(pretrained=True)

# Freeze the pre-trained weights
for param in model.parameters():
    param.requires_grad = False

# Modify the classifier part of AlexNet for SVHN
model.classifier = nn.Sequential(
    nn.Dropout(),
    nn.Linear(256 * 6 * 6, 4096),
    nn.ReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(4096, 4096),
    nn.ReLU(inplace=True),
    nn.Linear(4096, 10)  # Output layer for SVHN dataset
)

# Transfer the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Initialize the criterion and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / 100}")
            running_loss = 0.0

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on the test set: {100 * correct / total}%")


Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat


/home/dse313/miniconda3/envs/dse316_assignment/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10], Step [100/1145], Loss: 2.218105666637421
Epoch [1/10], Step [200/1145], Loss: 2.110550717115402
Epoch [1/10], Step [300/1145], Loss: 1.951042778491974
Epoch [1/10], Step [400/1145], Loss: 1.7350592195987702
Epoch [1/10], Step [500/1145], Loss: 1.5222872257232667
Epoch [1/10], Step [600/1145], Loss: 1.4065490019321443
Epoch [1/10], Step [700/1145], Loss: 1.2889359718561173
Epoch [1/10], Step [800/1145], Loss: 1.2421942168474198
Epoch [1/10], Step [900/1145], Loss: 1.181700549721718
Epoch [1/10], Step [1000/1145], Loss: 1.1523692524433136
Epoch [1/10], Step [1100/1145], Loss: 1.095138350725174
Epoch [2/10], Step [100/1145], Loss: 1.0461846941709518
Epoch [2/10], Step [200/1145], Loss: 1.029409232735634
Epoch [2/10], Step [300/1145], Loss: 1.0402674329280854
Epoch [2/10], Step [400/1145], Loss: 1.0287544041872025
Epoch [2/10], Step [500/1145], Loss: 0.9794797849655151
Epoch [2/10], Step [600/1145], Loss: 1.009885831475258
Epoch [2/10], Step [700/1145], Loss: 0.99533542871475

# Training VGG-16 Model

In [7]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
from torchvision.models import vgg16

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations for the dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load the SVHN dataset
trainset = torchvision.datasets.SVHN(root='./data', split='train', download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)

# Load pre-trained VGG-16
vgg = vgg16(pretrained=True)
# Freeze the pre-trained weights
for param in vgg.parameters():
    param.requires_grad = False

# Modify the last layer to fit the number of classes in SVHN (10)
vgg.classifier[-1] = nn.Linear(4096, 10)

# Move the model to GPU if available
vgg.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg.parameters(), lr=0.001, momentum=0.9)

# Training loop
epochs = 10
for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        optimizer.zero_grad()
        
        outputs = vgg(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on the test set: {100 * correct / total}%")

Using downloaded and verified file: ./data/train_32x32.mat


/home/dse313/miniconda3/envs/dse316_assignment/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[1,   100] loss: 2.195
[1,   200] loss: 2.053
[1,   300] loss: 1.998
[1,   400] loss: 1.940
[1,   500] loss: 1.941
[1,   600] loss: 1.897
[1,   700] loss: 1.906
[1,   800] loss: 1.876
[1,   900] loss: 1.831
[1,  1000] loss: 1.880
[1,  1100] loss: 1.848
[1,  1200] loss: 1.831
[1,  1300] loss: 1.841
[1,  1400] loss: 1.798
[1,  1500] loss: 1.824
[1,  1600] loss: 1.823
[1,  1700] loss: 1.792
[1,  1800] loss: 1.818
[1,  1900] loss: 1.844
[1,  2000] loss: 1.816
[1,  2100] loss: 1.822
[1,  2200] loss: 1.791
[2,   100] loss: 1.794
[2,   200] loss: 1.774
[2,   300] loss: 1.783
[2,   400] loss: 1.772
[2,   500] loss: 1.754
[2,   600] loss: 1.743
[2,   700] loss: 1.788
[2,   800] loss: 1.786
[2,   900] loss: 1.779
[2,  1000] loss: 1.754
[2,  1100] loss: 1.779
[2,  1200] loss: 1.755
[2,  1300] loss: 1.788
[2,  1400] loss: 1.791
[2,  1500] loss: 1.762
[2,  1600] loss: 1.741
[2,  1700] loss: 1.747
[2,  1800] loss: 1.765
[2,  1900] loss: 1.745
[2,  2000] loss: 1.759
[2,  2100] loss: 1.725
[2,  2200] 

# Training ResNet-18 Model

In [8]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load SVHN dataset
train_dataset = datasets.SVHN(root='./data', split='train', download=True, transform=transform)
test_dataset = datasets.SVHN(root='./data', split='test', download=True, transform=transform)

# Define data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Load pretrained ResNet-18 model
model = models.resnet18(pretrained=True)

# Freeze pretrained weights
for param in model.parameters():
    param.requires_grad = False

# Modify the last fully connected layer for SVHN
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)  # 10 classes in SVHN

# Send model to device
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")
    
# Test the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy on test set: {accuracy:.2f}%")


Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat
Epoch [1/10], Loss: 2.0302
Epoch [2/10], Loss: 1.9585
Epoch [3/10], Loss: 1.9520
Epoch [4/10], Loss: 1.9532
Epoch [5/10], Loss: 1.9507
Epoch [6/10], Loss: 1.9486
Epoch [7/10], Loss: 1.9477
Epoch [8/10], Loss: 1.9505
Epoch [9/10], Loss: 1.9482
Epoch [10/10], Loss: 1.9476
Accuracy on test set: 33.04%


# Training ResNet-50 Model

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torchvision.transforms.functional as TF

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load SVHN dataset
train_data = datasets.SVHN(root='./data', split='train', download=True, transform=transform)
test_data = datasets.SVHN(root='./data', split='test', download=True, transform=transform)

# Data loaders
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# Load pre-trained ResNet50 model
model = models.resnet50(pretrained=True)

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

# Replace the final fully connected layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)  # SVHN has 10 classes

# Move model to device
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct / total

    print(f'Epoch [{epoch + 1}/{num_epochs}], '
          f'Train Loss: {train_loss:.4f}, '
          f'Train Accuracy: {train_accuracy:.2f}%')

# Test the model
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f'Test Accuracy: {test_accuracy:.2f}%')


Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat


/home/dse313/miniconda3/envs/dse316_assignment/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10], Train Loss: 1.8382, Train Accuracy: 36.03%
Epoch [2/10], Train Loss: 1.7005, Train Accuracy: 41.52%
Epoch [3/10], Train Loss: 1.6560, Train Accuracy: 43.05%
Epoch [4/10], Train Loss: 1.6263, Train Accuracy: 44.24%
Epoch [5/10], Train Loss: 1.5950, Train Accuracy: 45.58%
Epoch [6/10], Train Loss: 1.5788, Train Accuracy: 46.06%
Epoch [7/10], Train Loss: 1.5642, Train Accuracy: 46.62%
Epoch [8/10], Train Loss: 1.5455, Train Accuracy: 47.34%
Epoch [9/10], Train Loss: 1.5333, Train Accuracy: 47.56%
Epoch [10/10], Train Loss: 1.5209, Train Accuracy: 48.14%
Test Accuracy: 48.78%


# Training ResNet-101 Model

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms, models

# Set device to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define data transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to fit ResNet input size
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize images
])

# Load SVHN dataset
train_dataset = datasets.SVHN(root='./data', split='train', download=True, transform=transform)
test_dataset = datasets.SVHN(root='./data', split='test', download=True, transform=transform)

# Define data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Load pre-trained ResNet-101 model
model = models.resnet101(pretrained=True)

# Freeze all layers except the final fully connected layer
for param in model.parameters():
    param.requires_grad = False
model.fc = nn.Linear(model.fc.in_features, 10)  # Change the output layer to fit SVHN dataset

# Move model to device
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy on the test set: {accuracy:.2f}%")


Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat


/home/dse313/miniconda3/envs/dse316_assignment/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10], Loss: 1.8318
Epoch [2/10], Loss: 1.7034
Epoch [3/10], Loss: 1.6706
Epoch [4/10], Loss: 1.6475
Epoch [5/10], Loss: 1.6172
Epoch [6/10], Loss: 1.6018
Epoch [7/10], Loss: 1.5826
Epoch [8/10], Loss: 1.5768
Epoch [9/10], Loss: 1.5573
Epoch [10/10], Loss: 1.5475
Accuracy on the test set: 47.69%
